In [70]:
# import our usual things
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
import matplotlib.pyplot as plt

In [71]:
buildings = pd.read_csv('building_inventory.csv',
               na_values={'Square Footage': 0, 
                         'Year Acquired': 0,
                         'Year Constructed': 0, 
                         'Floors':0})

In [72]:
buildings_heatmap = pd.pivot_table(buildings,
                                       index = ['Congress Dist'],
                                       values = ['Square Footage'],
                                       columns = ['Agency Name'],
                                       aggfunc = np.sum)

In [73]:
buildings_heatmap

Square Footage  \
Agency Name   Appellate Court / Fifth District   
Congress Dist                                    
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                     15124.0   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   

                                                 \
Agency Name   Appellate Court / Fourth District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                           NaN   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                      16400.0   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                 \
Agency Name   Appellate Court / Second District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                       43330.0   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                          NaN   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                                         \
Agency Name   Appellate Court / Third District Chicago State University   
Congress Dist                                                             
0                                          NaN                      NaN   
1                                          NaN                1219492.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
5                                          NaN                      NaN   
6         

In [74]:
mySelectedLabel = ipywidgets.Label()

In [75]:
Agency_Name = np.asarray(buildings_heatmap.columns.levels[1].to_list())
Congress_Dist = np.asarray(buildings_heatmap.index.to_list())
Square_Footage = buildings_heatmap.values

In [76]:
# heatmap
col_sc = bqplot.ColorScale(scheme="RdPu", min=np.nanmin(buildings_heatmap), max=np.nanmax(buildings_heatmap)) 
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

ax_col = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

ax_x = bqplot.Axis(scale = x_sc, label='Agency Names')
ax_y = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Congress Dist')

heat_map = bqplot.GridHeatMap(color = buildings_heatmap,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

In [77]:
total_year = buildings.groupby("Year Acquired")["Square Footage"].sum()

In [78]:
total_year

Year Acquired
1753.0      1200.0
1802.0      4440.0
1810.0      4033.0
1832.0    120000.0
1837.0     10302.0
            ...   
2015.0    305093.0
2016.0    304839.0
2017.0      6720.0
2018.0     17160.0
2019.0      1520.0
Name: Square Footage, Length: 171, dtype: float64

In [83]:
# line graph
i,j = 17, 6

Name = [Agency_Name[j]] # min/max longitude
Dist = [Congress_Dist[i],Congress_Dist[i+1]] # min/max latitude
mask_name = []

# do a mask
for i in range(len(buildings['Agency Name'])):
    mask_name.append(Name[0])

name_mask = (buildings['Agency Name'] == mask_name)
region_mask = ( (buildings['Congress Dist'] >= Dist[0]) & (buildings['Congress Dist'] < Dist[1]))
final_mask = name_mask & region_mask

mask_buildings = buildings[final_mask]
sorted_mask_buildings = mask_buildings.sort_values('Year Acquired')
agg_building = sorted_mask_buildings.groupby("Year Acquired")["Square Footage"].sum()

# plot line graph
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

lines = bqplot.Lines(x = sorted_mask_buildings['Year Acquired'].unique(), y = agg_building, scales = {'x': x_sc, 'y': y_sc})
ax_x = bqplot.Axis(scale = x_sc, label = 'Year Acquired')
ax_y = bqplot.Axis(scale = y_sc, label = 'Total Square Footage', orientation = 'vertical')

In [87]:
# interactivity
def get_data_value(change):
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        l = buildings_heatmap.iloc[i,j]
        mySelectedLabel.value = 'Total Square Footage = ' + str(l)  + ', Agency: ' + Agency_Name[j] + ', Congress District: ' + str(i)
        
        #line graph
        Name = [Agency_Name[j]] 
        if i == Congress_Dist[-1]:
            Dist = [Congress_Dist[i],Congress_Dist[i]+1]
        else:
            Dist = [Congress_Dist[i],Congress_Dist[i+1]]
        mask_name = []

        #do a mask
        for i in range(len(buildings['Agency Name'])):
            mask_name.append(Name[0])

        name_mask = (buildings['Agency Name'] == mask_name)
        region_mask = ( (buildings['Congress Dist'] >= Dist[0]) & (buildings['Congress Dist'] < Dist[1]))
        final_mask = name_mask & region_mask
        
        # make the data plot following the requirement
        mask_buildings = buildings[final_mask]
        sorted_mask_buildings = mask_buildings.sort_values('Year Acquired')
        agg_building = sorted_mask_buildings.groupby("Year Acquired")["Square Footage"].sum()
        lines.x = sorted_mask_buildings['Year Acquired'].unique()
        lines.y = agg_building

heat_map.observe(get_data_value, 'selected')

In [88]:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [ax_col, ax_y, ax_x])
fig_line = bqplot.Figure(marks = [lines], axes = [ax_x, ax_y])

In [89]:
# display dashboard for the buildings data
fig_heatmap.layout.min_width = '500px'
fig_line.layout.min_width = '500px'

myDashboard = ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_heatmap,fig_line])])
myDashboard

In [ ]:
## The data transformations or rescalings I did for the plot is for heatmap.

## For NaN's (empty entries) in the dataset, as a special values defined in numpy, return min or max of an array or minimum along an axis

## The aesthetic choices that I made (colors, layout, plot size, label size) is just following the example that the 
## in class notebook showed, not much changed. I was trying to import "matplotlib.colors" but failed.